In [6]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [7]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [8]:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

In [9]:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

In [10]:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

In [11]:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

In [12]:
df = df.drop(df[(df.Borough == "Not assigned")].index)

In [13]:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

In [14]:
df.Neighbourhood.fillna(df.Borough, inplace=True)

In [15]:
df=df.drop_duplicates()

In [16]:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))
df.update(
    df.Borough.loc[
       lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

In [17]:
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes
df2.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park (Toronto)
5,M9A,Queen's Park (Toronto),Queen's Park (Toronto)
6,M1B,"Scarborough, Toronto","Malvern, Toronto, Rouge, Toronto"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [18]:
df2.shape

(103, 3)

In [21]:
 from geopy.geocoders import Nominatim

ModuleNotFoundError: No module named 'geopy'

In [20]:
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park (Toronto),43.662301,-79.389494
5,M9A,Queen's Park (Toronto),Queen's Park (Toronto),43.667856,-79.532242
6,M1B,"Scarborough, Toronto","Malvern, Toronto, Rouge, Toronto",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [22]:
geo_data=toronto_data[toronto_data['Borough'].str.contains("Toronto")]
geo_data.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park (Toronto),43.662301,-79.389494
5,M9A,Queen's Park (Toronto),Queen's Park (Toronto),43.667856,-79.532242
6,M1B,"Scarborough, Toronto","Malvern, Toronto, Rouge, Toronto",43.806686,-79.194353
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
12,M1C,"Scarborough, Toronto","Highland Creek (Toronto), Port Union, Toronto,...",43.784535,-79.160497
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
18,M1E,"Scarborough, Toronto","West Hill, Toronto, Morningside, Toronto, Guil...",43.763573,-79.188711
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [23]:
CLIENT_ID = '2K5LH2ZGIJBRCOZOFWPIPWT0YURQZRE400ZOZQOMKPSL2UIA' # your Foursquare ID
CLIENT_SECRET = 'FC0AG5Q30I3GIWXU25II3WQQPVID4LZAWAZBXP5GAXCMWWJ5' # your Foursquare Secret
VERSION = '20180604'

In [25]:
    
  
 def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park (Toronto)
Queen's Park (Toronto)
Malvern, Toronto, Rouge, Toronto
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Toronto, Cloverdale
Highland Creek (Toronto), Port Union, Toronto, Rouge Hill
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Markland Wood, Old Burnhamthorpe, Bloordale Gardens
West Hill, Toronto, Morningside, Toronto, Guildwood
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn, Toronto
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Downsview North, Wilson Heights, Toronto, Bathurst Manor
Thorncliffe Park
Adelaide, Richmond, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Oriole, Henry Farm
York University, Northwood Park
East Toronto
Harbourfront East, Toronto Islands, Union Station (Toronto)
Little Portugal, Toronto

In [27]:
toronto_venues.head(12)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Regent Park,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
7,Regent Park,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,Regent Park,43.654260,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
9,Regent Park,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, Richmond, King",100,100,100,100,100,100
"Agincourt, Toronto",5,5,5,5,5,5
"Alderwood, Toronto, Long Branch, Toronto",9,9,9,9,9,9
Bayview Village,4,4,4,4,4,4
Berczy Park,55,55,55,55,55,55
...,...,...,...,...,...,...
"Woburn, Toronto",3,3,3,3,3,3
Woodbine Heights,8,8,8,8,8,8
York Mills West,4,4,4,4,4,4


In [29]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(2217, 270)

In [30]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, Richmond, King",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.020000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,"Agincourt, Toronto",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Alderwood, Toronto, Long Branch, Toronto",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.018182,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(12)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Steakhouse,Café,Bar,Bakery,Cosmetics Shop,Restaurant,Asian Restaurant,Thai Restaurant,Pizza Place
1,"Agincourt, Toronto",Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Alderwood, Toronto, Long Branch, Toronto",Pizza Place,Coffee Shop,Pool,Skating Rink,Pharmacy,Gym,Pub,Sandwich Place,Dance Studio,Deli / Bodega
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar,Café,Seafood Restaurant,Steakhouse,Farmers Market,Shopping Mall
5,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Recording Studio,Skate Park,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Spa,Garden Center
6,"CFB Toronto, Downsview East",Park,Airport,Snack Place,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
7,"Cabbagetown, Toronto, St. James Town",Coffee Shop,Park,Restaurant,Italian Restaurant,Bakery,Café,Pub,Market,Pizza Place,Chinese Restaurant
8,Caledonia-Fairbanks,Women's Store,Park,Fast Food Restaurant,Market,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Sandwich Place,Gym,Fried Chicken Joint,Middle Eastern Restaurant,American Restaurant,Burrito Place,Yoga Studio,Dessert Shop


In [33]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
from bs4 import BeautifulSoup
import lxml
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Restaurant,Beer Store,Breakfast Spot,Shoe Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Carpet Store,Event Space,Boutique,Coffee Shop,Miscellaneous Shop,Vietnamese Restaurant,Furniture / Home Store,Dessert Shop
4,M7A,Downtown Toronto,Queen's Park (Toronto),43.662301,-79.389494,0.0,Coffee Shop,Park,Gym,Yoga Studio,Restaurant,Beer Bar,Italian Restaurant,Seafood Restaurant,Sandwich Place,Burger Joint


In [35]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, Richmond, King",Coffee Shop,Steakhouse,Café,Bar,Bakery,Cosmetics Shop,Restaurant,Asian Restaurant,Thai Restaurant,Pizza Place
1,0,"Agincourt, Toronto",Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,0,"Alderwood, Toronto, Long Branch, Toronto",Pizza Place,Coffee Shop,Pool,Skating Rink,Pharmacy,Gym,Pub,Sandwich Place,Dance Studio,Deli / Bodega
3,0,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar,Café,Seafood Restaurant,Steakhouse,Farmers Market,Shopping Mall


In [36]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

NameError: name 'Nominatim' is not defined

In [37]:
import folium
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
       
map_clusters

ModuleNotFoundError: No module named 'folium'